In [1]:
import numpy as np
np.set_printoptions(precision=3)
np.set_printoptions(suppress=True)
np.set_printoptions(linewidth=60)

import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib import style
style.use('ggplot')
import itertools

from stanhelper import stanhelper

%matplotlib inline
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os
import sys

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import numpy.random as npr
from sklearn.cluster import KMeans
from scipy.stats import invgamma

from scipy import sparse, stats

plt.style.use('ggplot')

In [2]:
import random
import time
import tensorflow as tf
import edward as ed
randseed = 29266137
# randseed = int(time.time()*1000000%100000000)
print("random seed: ", randseed)
random.seed(randseed)
np.random.seed(randseed)
tf.set_random_seed(randseed)

/Users/yixinwang/py2/lib/python2.7/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


random seed:  29266137


In [3]:
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

In [4]:
import pmf

In [5]:
OUT_DATA_DIR = '../dat/tmdb_proc/'
OUT_DIR = '../res/factorfit/'

# load causes

In [6]:
casts = pd.read_csv('../dat/tmdb_proc/casts_df_clean.csv')

In [7]:
data = pd.read_csv('../dat/tmdb_raw/tmdb-5000-movie-dataset/tmdb_5000_movies.csv')

In [8]:
casts.head()

,Unnamed: 0,movie_id,cast_id,character,credit_id,gender,actor_id,name,order,new_actor_id,new_movie_id
0,0,19995,242,Jake Sully,5602a8a7c3a3685532001c9a,2,65731,Sam Worthington,0,0,1646
1,1,19995,3,Neytiri,52fe48009251416c750ac9cb,1,8691,Zoe Saldana,1,1,1646
2,2,19995,25,Dr. Grace Augustine,52fe48009251416c750aca39,1,10205,Sigourney Weaver,2,2,1646
3,3,19995,5,Trudy Chacon,52fe48009251416c750ac9d3,1,17647,Michelle Rodriguez,4,3,1646
4,4,19995,8,Selfridge,52fe48009251416c750ac9e1,2,1771,Giovanni Ribisi,5,4,1646


In [9]:
ratings = casts[['actor_id', 'movie_id']]
# ratings['new_actor_id'] +=1
# ratings['new_movie_id'] +=1

In [10]:
ratings.columns = ['songId','userId']

In [11]:
# ratings=ratings.set_index('userId')
ratings['rating'] = 1

/Users/yixinwang/py2/lib/python2.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [12]:
ratings.head()

,songId,userId,rating
0,65731,19995,1
1,8691,19995,1
2,10205,19995,1
3,17647,19995,1
4,1771,19995,1


In [13]:
tr_vd_data = ratings

In [14]:
tr_vd_data.head(), tr_vd_data.shape

(   songId  userId  rating
 0   65731   19995       1
 1    8691   19995       1
 2   10205   19995       1
 3   17647   19995       1
 4    1771   19995       1, (14023, 3))

In [15]:
def split_train_test_proportion(data, uid, test_prop=0.5, random_seed=0):
    data_grouped_by_user = data.groupby(uid)
    tr_list, te_list = list(), list()

    np.random.seed(random_seed)

    for u, (_, group) in enumerate(data_grouped_by_user):
        n_items_u = len(group)

        if n_items_u >= 5:
            idx = np.zeros(n_items_u, dtype='bool')
            idx[np.random.choice(n_items_u, size=int(test_prop * n_items_u), replace=False).astype('int64')] = True

            tr_list.append(group[np.logical_not(idx)])
            te_list.append(group[idx])
        else:
            tr_list.append(group)

        if u % 5000 == 0:
            print("%d users sampled" % u)
            sys.stdout.flush()

    data_tr = pd.concat(tr_list)
    data_te = pd.concat(te_list)
    
    return data_tr, data_te

In [16]:
def get_count(tp, id):
    playcount_groupbyid = tp[[id]].groupby(id, as_index=False)
    count = playcount_groupbyid.size()
    return count

In [17]:
user_activity = get_count(tr_vd_data, 'userId')
item_popularity = get_count(tr_vd_data, 'songId')

In [18]:
unique_uid = user_activity.index
unique_sid = item_popularity.index

In [19]:
n_users = len(unique_uid)
n_items = len(unique_sid)

In [20]:
n_users, n_items

(2828, 901)

In [21]:
song2id = dict((sid, i) for (i, sid) in enumerate(unique_sid))
user2id = dict((uid, i) for (i, uid) in enumerate(unique_uid))
id2user = dict((i, uid) for (i, uid) in enumerate(unique_uid))
id2song = dict((i, sid) for (i, sid) in enumerate(unique_sid))

In [22]:
with open(os.path.join(OUT_DATA_DIR, 'unique_uid.txt'), 'w') as f:
    for uid in unique_uid:
        f.write('%s\n' % uid)

with open(os.path.join(OUT_DATA_DIR, 'unique_sid.txt'), 'w') as f:
    for sid in unique_sid:
        f.write('%s\n' % sid)

In [23]:
def numerize(tp):
    uid = list(map(lambda x: user2id[x], tp['userId']))
    sid = list(map(lambda x: song2id[x], tp['songId']))
    tp.loc[:, 'uid'] = uid
    tp.loc[:, 'sid'] = sid
    return tp[['uid', 'sid', 'rating']]

In [24]:
tr_vd_data = numerize(tr_vd_data)
# test_data = numerize(test_data)

/Users/yixinwang/py2/lib/python2.7/site-packages/pandas/core/indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/Users/yixinwang/py2/lib/python2.7/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [25]:
train_data, vad_data = split_train_test_proportion(tr_vd_data, 'uid', test_prop=0.2, random_seed=12345)

0 users sampled


In [26]:
print("There are total of %d unique users in the training set and %d unique users in the entire dataset" % \
(len(pd.unique(train_data['uid'])), len(unique_uid)))

There are total of 2828 unique users in the training set and 2828 unique users in the entire dataset


In [27]:
def move_to_fill(part_data_1, part_data_2, unique_id, key):
    # move the data from part_data_2 to part_data_1 so that part_data_1 has the same number of unique "key" as unique_id
    part_id = set(pd.unique(part_data_1[key]))
    
    left_id = list()
    for i, _id in enumerate(unique_id):
        if _id not in part_id:
            left_id.append(_id)
            
    move_idx = part_data_2[key].isin(left_id)
    part_data_1 = part_data_1.append(part_data_2[move_idx])
    part_data_2 = part_data_2[~move_idx]
    return part_data_1, part_data_2

In [28]:
train_data, vad_data = move_to_fill(train_data, vad_data, np.arange(n_items), 'sid')

In [29]:
train_data.to_csv(os.path.join(OUT_DATA_DIR, 'train.csv'), index=False)
vad_data.to_csv(os.path.join(OUT_DATA_DIR, 'validation.csv'), index=False)
tr_vd_data.to_csv(os.path.join(OUT_DATA_DIR, 'train_full.csv'), index=False)

In [30]:
unique_uid = list()
with open(os.path.join(OUT_DATA_DIR, 'unique_uid.txt'), 'r') as f:
    for line in f:
        unique_uid.append(line.strip())

unique_sid = list()
with open(os.path.join(OUT_DATA_DIR, 'unique_sid.txt'), 'r') as f:
    for line in f:
        unique_sid.append(line.strip())

n_users = len(unique_uid)
n_items = len(unique_sid)

In [31]:
def load_data(csv_file, shape=(n_users, n_items)):
    tp = pd.read_csv(csv_file)
    rows, cols, vals = np.array(tp['uid']), np.array(tp['sid']), np.array(tp['rating']) 
    data = sparse.csr_matrix((vals, (rows, cols)), dtype=np.float32, shape=shape)
    return data


train_data = load_data(os.path.join(OUT_DATA_DIR, 'train.csv'))
vad_data = load_data(os.path.join(OUT_DATA_DIR, 'validation.csv'))

# PMF substitute confounder

In [32]:
train_data_imp = train_data
vad_data_imp = vad_data

In [33]:
dims = np.array([50])

In [34]:
train_data_coo = train_data_imp.tocoo()
row_tr, col_tr = train_data_coo.row, train_data_coo.col

vad_data_coo = vad_data_imp.tocoo()
row_vd, col_vd = vad_data_coo.row, vad_data_coo.col

for i, dim in enumerate(dims):
    print("dim", dim)
    pf = pmf.PoissonMF(n_components=dim, random_state=98765, verbose=True, a=0.3, b=0.3, c=0.3, d=0.3)
    pf.fit(train_data, row_tr, col_tr, dict(X_new=vad_data.data, rows_new=row_vd, cols_new=col_vd))
    U, V = pf.Eb, pf.Et.T
    np.savetxt(OUT_DIR + '/cause_pmf_k'+str(dim)+'_U.csv', U)
    np.savetxt(OUT_DIR + '/cause_pmf_k'+str(dim)+'_V.csv', V)



dim 50
ITERATION: 0	Pred_ll: -4.69	Old Pred_ll: -inf	Improvement: nan
ITERATION: 1	Pred_ll: -4.55	Old Pred_ll: -4.69	Improvement: 0.02861
ITERATION: 2	Pred_ll: -4.50	Old Pred_ll: -4.55	Improvement: 0.01118
ITERATION: 3	Pred_ll: -4.48	Old Pred_ll: -4.50	Improvement: 0.00523
ITERATION: 4	Pred_ll: -4.45	Old Pred_ll: -4.48	Improvement: 0.00500


pmf.py:150: RuntimeWarning: invalid value encountered in double_scalars
  improvement = (pred_ll - old_pll) / abs(old_pll)


ITERATION: 5	Pred_ll: -4.41	Old Pred_ll: -4.45	Improvement: 0.00908
ITERATION: 6	Pred_ll: -4.39	Old Pred_ll: -4.41	Improvement: 0.00487
ITERATION: 7	Pred_ll: -4.39	Old Pred_ll: -4.39	Improvement: 0.00085
ITERATION: 8	Pred_ll: -4.39	Old Pred_ll: -4.39	Improvement: 0.00043
ITERATION: 9	Pred_ll: -4.38	Old Pred_ll: -4.39	Improvement: 0.00057
ITERATION: 10	Pred_ll: -4.38	Old Pred_ll: -4.38	Improvement: 0.00047
ITERATION: 11	Pred_ll: -4.38	Old Pred_ll: -4.38	Improvement: 0.00035
ITERATION: 12	Pred_ll: -4.38	Old Pred_ll: -4.38	Improvement: 0.00021
ITERATION: 13	Pred_ll: -4.38	Old Pred_ll: -4.38	Improvement: 0.00014
ITERATION: 14	Pred_ll: -4.38	Old Pred_ll: -4.38	Improvement: 0.00014
ITERATION: 15	Pred_ll: -4.38	Old Pred_ll: -4.38	Improvement: 0.00006


In [35]:
# illustrate what each dimension of the pmf substitute confounders mean
print("Which movies has the k-th latent dimension on? \
(The latent dimensions capture movie themes.)")
num_top = 20
for K in range(50):
    topids = U[:,K].argsort()[-num_top:][::-1]
    print("\n\n\nK = ", K)
    print([(i, data[data["id"]==int(id2user[topid])]["title"]) \
           for i, topid in enumerate(topids)])

Which movies has the k-th latent dimension on? (The latent dimensions capture movie themes.)



K =  0
[(0, 284    Little Fockers
Name: title, dtype: object), (1, 1710    The Royal Tenenbaums
Name: title, dtype: object), (2, 811    Meet the Parents
Name: title, dtype: object), (3, 3006    Twilight Zone: The Movie
Name: title, dtype: object), (4, 898    The Twilight Saga: New Moon
Name: title, dtype: object), (5, 172    The Twilight Saga: Breaking Dawn - Part 2
Name: title, dtype: object), (6, 2919    My Cousin Vinny
Name: title, dtype: object), (7, 1636    Fled
Name: title, dtype: object), (8, 2950    The Perez Family
Name: title, dtype: object), (9, 2225    Inherent Vice
Name: title, dtype: object), (10, 3766    Four Rooms
Name: title, dtype: object), (11, 688    Meet the Fockers
Name: title, dtype: object), (12, 2181    From Dusk Till Dawn
Name: title, dtype: object), (13, 1845    Hide and Seek
Name: title, dtype: object), (14, 1178    Vantage Point
Name: title, dtype: object), (15, 

[(0, 1363    Spy Kids
Name: title, dtype: object), (1, 1155    Spy Kids 3-D: Game Over
Name: title, dtype: object), (2, 238    Teenage Mutant Ninja Turtles
Name: title, dtype: object), (3, 2181    From Dusk Till Dawn
Name: title, dtype: object), (4, 1302    Spy Kids 2: The Island of Lost Dreams
Name: title, dtype: object), (5, 241    Teenage Mutant Ninja Turtles: Out of the Shadows
Name: title, dtype: object), (6, 1116    The Firm
Name: title, dtype: object), (7, 942    The Book of Life
Name: title, dtype: object), (8, 1495    Phenomenon
Name: title, dtype: object), (9, 880    Grindhouse
Name: title, dtype: object), (10, 633    Lincoln
Name: title, dtype: object), (11, 3734    The Crying Game
Name: title, dtype: object), (12, 1270    Universal Soldier: The Return
Name: title, dtype: object), (13, 9    Batman v Superman: Dawn of Justice
Name: title, dtype: object), (14, 281    American Gangster
Name: title, dtype: object), (15, 2178    Into the Wild
Name: title, dtype: object), (16, 151

[(0, 1532    The Grand Budapest Hotel
Name: title, dtype: object), (1, 243    Night at the Museum
Name: title, dtype: object), (2, 195    Night at the Museum: Secret of the Tomb
Name: title, dtype: object), (3, 720    Contagion
Name: title, dtype: object), (4, 2461    Moonrise Kingdom
Name: title, dtype: object), (5, 1531    Moonraker
Name: title, dtype: object), (6, 501    The Little Prince
Name: title, dtype: object), (7, 751    Duplicity
Name: title, dtype: object), (8, 1257    Fantastic Mr. Fox
Name: title, dtype: object), (9, 3075    Crash
Name: title, dtype: object), (10, 3884    Goldfinger
Name: title, dtype: object), (11, 3143    You Only Live Twice
Name: title, dtype: object), (12, 4071    From Russia with Love
Name: title, dtype: object), (13, 3162    Thunderball
Name: title, dtype: object), (14, 3336    Diamonds Are Forever
Name: title, dtype: object), (15, 3351    The Man with the Golden Gun
Name: title, dtype: object), (16, 1713    For Your Eyes Only
Name: title, dtype: ob




K =  28
[(0, 33    X-Men: The Last Stand
Name: title, dtype: object), (1, 46    X-Men: Days of Future Past
Name: title, dtype: object), (2, 511    X-Men
Name: title, dtype: object), (3, 3053    Movie 43
Name: title, dtype: object), (4, 203    X2
Name: title, dtype: object), (5, 232    The Wolverine
Name: title, dtype: object), (6, 441    Robots
Name: title, dtype: object), (7, 468    Swordfish
Name: title, dtype: object), (8, 122    X-Men Origins: Wolverine
Name: title, dtype: object), (9, 312    Green Zone
Name: title, dtype: object), (10, 3406    Dazed and Confused
Name: title, dtype: object), (11, 1612    Larry Crowne
Name: title, dtype: object), (12, 1921    Michael Collins
Name: title, dtype: object), (13, 1759    The Right Stuff
Name: title, dtype: object), (14, 431    Charlotte's Web
Name: title, dtype: object), (15, 1123    Boomerang
Name: title, dtype: object), (16, 1046    The Pelican Brief
Name: title, dtype: object), (17, 1182    Big Momma's House 2
Name: title, dtype: o

[(0, 1091    Nixon
Name: title, dtype: object), (1, 2496    Soul Plane
Name: title, dtype: object), (2, 1752    Kiss the Girls
Name: title, dtype: object), (3, 1924    Abandon
Name: title, dtype: object), (4, 401    Enemy at the Gates
Name: title, dtype: object), (5, 2168    Appaloosa
Name: title, dtype: object), (6, 1759    The Right Stuff
Name: title, dtype: object), (7, 778    Meet Dave
Name: title, dtype: object), (8, 1792    Pain & Gain
Name: title, dtype: object), (9, 353    Tropic Thunder
Name: title, dtype: object), (10, 1146    Man on a Ledge
Name: title, dtype: object), (11, 1221    The Doors
Name: title, dtype: object), (12, 2301    Take Me Home Tonight
Name: title, dtype: object), (13, 1836    Tombstone
Name: title, dtype: object), (14, 94    Guardians of the Galaxy
Name: title, dtype: object), (15, 189    Noah
Name: title, dtype: object), (16, 80    Snow White and the Huntsman
Name: title, dtype: object), (17, 2477    Jennifer's Body
Name: title, dtype: object), (18, 1630 

# PCA substitute confounder

In [36]:
dim = 50
X = train_data_coo.todense()
Xscaler = preprocessing.StandardScaler(with_std=False).fit(X)
Xscaled = Xscaler.fit_transform(X)
pca = PCA(n_components=dim)
pca.fit(Xscaled)
pcaU = pca.fit_transform(Xscaled)
np.savetxt(OUT_DIR + '/cause_pca_k'+str(dim)+'_U.csv', pcaU)
np.savetxt(OUT_DIR + '/cause_pca_k'+str(dim)+'_V.csv', pca.components_.T)

# DEF substitute confounder

In [37]:
from fit_def import fit_def

fit_def.py:11: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

The backend was *originally* set to 'module://ipykernel.pylab.backend_inline' by the following code:
  File "/System/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/runpy.py", line 174, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/System/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/Users/yixinwang/py2/lib/python2.7/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/Users/yixinwang/py2/lib/python2.7/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/Users/yixinwang/py2/lib/python2.7/site-packages/ipykernel/kernelapp.py", lin

In [38]:
def_x_post, \
def_z3_post, def_z2_post, def_z1_post, \
def_W0_post, def_x_post_np, def_z_post_np = \
fit_def(X.T, K=[50,20,5], prior_a=0.1, prior_b=100., \
        optimizer=tf.train.AdamOptimizer(1e-3), n_iter=100000)

Epoch 0
1000/1000 [100%] ██████████████████████████████ Elapsed: 23s
Negative log-likelihood <= 5705453.819
Perplexity <= 151894012843531494445974496229055017569493080983418713940647207329563001767104774136107828805866257994556508838645957687822855823776855046850200894710315651322381868111338820970290624351915292776988672.000
trivial mse 0.0048704334
mse 0.004879967
5705453.81875 1e+16 True
cur-LL 5705453.81875 min-LL 5705453.81875 diffratio-LL 0.0
-1e+16 0.0
Epoch 1
1000/1000 [100%] ██████████████████████████████ Elapsed: 28s
Negative log-likelihood <= 2791740.347
Perplexity <= 89329711001902319999884979184727660036951572998080608662454334163968749804998119773415835871215616.000
trivial mse 0.0048704334
mse 0.0048700906
2791740.347 5705453.81875 True
cur-LL 2791740.347 min-LL 2791740.347 diffratio-LL 0.0
0.0 0.0
Epoch 2
1000/1000 [100%] ██████████████████████████████ Elapsed: 28s
Negative log-likelihood <= 1749721.890
Perplexity <= 24588868702842171737808056045397562864860201884280017

1000/1000 [100%] ██████████████████████████████ Elapsed: 23s
Negative log-likelihood <= 55946.848
Perplexity <= 91.823
trivial mse 0.0048704334
mse 0.0048704306
55946.847890625 59171.300375 True
cur-LL 55946.847890625 min-LL 55946.847890625 diffratio-LL 0.0
0.0 0.0
Epoch 29
1000/1000 [100%] ██████████████████████████████ Elapsed: 27s
Negative log-likelihood <= 53084.764
Perplexity <= 72.867
trivial mse 0.0048704334
mse 0.004870432
53084.76378125 55946.847890625 True
cur-LL 53084.76378125 min-LL 53084.76378125 diffratio-LL 0.0
0.0 0.0
Epoch 30
1000/1000 [100%] ██████████████████████████████ Elapsed: 26s
Negative log-likelihood <= 50489.473
Perplexity <= 59.084
trivial mse 0.0048704334
mse 0.004870427
50489.473234375 53084.76378125 True
cur-LL 50489.473234375 min-LL 50489.473234375 diffratio-LL 0.0
0.0 0.0
Epoch 31
1000/1000 [100%] ██████████████████████████████ Elapsed: 25s
Negative log-likelihood <= 48144.710
Perplexity <= 48.888
trivial mse 0.0048704334
mse 0.0048704296
48144.71040625

1000/1000 [100%] ██████████████████████████████ Elapsed: 27s
Negative log-likelihood <= 29157.841
Perplexity <= 10.545
trivial mse 0.0048704334
mse 0.0048704334
29157.840953125 29126.161484375 False
cur-LL 29157.840953125 min-LL 29126.161484375 diffratio-LL 0.0010876637062867097
0.0 0.0010876637062867097
Epoch 59
1000/1000 [100%] ██████████████████████████████ Elapsed: 25s
Negative log-likelihood <= 29215.496
Perplexity <= 10.594
trivial mse 0.0048704334
mse 0.0048704334
29215.496453125 29126.161484375 False
cur-LL 29215.496453125 min-LL 29126.161484375 diffratio-LL 0.0030671727477005802
0.0010876637062867097 0.0030671727477005802


In [39]:
np.savetxt(OUT_DIR + '/cause_def_k_U.csv', def_z_post_np.T)
np.savetxt(OUT_DIR + '/cause_def_k_Xpost.csv', def_x_post_np.T)